# setup 


In [41]:
# simple 101 group
F101 = IntegerModRing(101)

In [42]:
# define elliptic curve with coefficient 0(degree 1), 3(degree 0)
E = EllipticCurve(F101, [0,3])

In [43]:
# define polynomial ring on F101, then extended in Galois field
R.<X> = PolynomialRing(F101)
K.<X> = GF(101**2, modulus = x^2+2)
E2 = EllipticCurve(K, [0,3])
# EC generator at  (1,2)
G=E2([1,2])

In [44]:
# the order of G is r=17, the embedding degree it's the smallest k such that r|p^k -1, thus k=2
## if you keep adding G to itself, it would get only 17 unique values r=17
## group base p=101
G2 = E2([36,31*X])
# structured reference string SRS
## in kate commitment, here you create SRS, and release the toxic waste
## for random value rand=2
rand=2
G1_SRS = [(rand**i)*G for i in range(7)]
G2_SRS = [rand**i*G2 for i in range(2)]

In [45]:
N=G.order()
F17=Integers(N)
F17.square_roots_of_one()
# witness values for random [1,4,-1,-4]
w = vector(F17, [1,4,-1,-4])
w
k1=2; k2=3
k1*w
k2*w
A = matrix(F17, [[1**i for i in range (4)], [4**i for i in range (4)], [16**i for i in range(4)], [13**i for i in range(4)]])
Ai = A.inverse()

In [46]:
s=2
zeta=5
beta=12
gamma=13
vega=12
alpha = 15
ql_coef = [0,0,1,0] # multiplication a1*b1 : q left mulitplication coefficient for l+ r + m + c  = rhs
qr_coef = [0,0,1,0] # multiplication a2*b2'
qm_coef = [1,1,0,0] # addition gate a3+b3
qo_coef = [-1,-1,0,0] # ??
qc_coef = [0,0,-30,0] #??
P.<x> = F17[];x=P.0
ql = P(list(Ai*vector(F17, ql_coef)))
qr = P(list(Ai*vector(F17, qr_coef)))
qm = P(list(Ai*vector(F17, qm_coef)))
qo = P(list(Ai*vector(F17, qo_coef)))
qc = P(list(Ai*vector(F17, qc_coef)))
#public setup polynomials SRS are ql,qr,qm,qo,qc
ql,qr,qm,qo,qc
#
# equality constraints ??
sa = P(list(Ai*vector(F17, [2,3,12,13]))) # ??
sb = P(list(Ai*vector(F17, [1,15,8,9]))) # ??
sc = P(list(Ai*vector(F17, [4,16,14,5]))) # ?? 
sa,sb,sc
# blinding values bld1-bld9
bld1=7
bld2=4
bld3=11
bld4=12
bld5=16
bld6=2
bld7=14
bld8=2
bld9=11

# proving

In [57]:
def proving():
        ## for root 3, the following toxic waste coefficient are:
        a_coef = [3,9,27,0] # the left operands: a1,a2,a3,a5 
        b_coef = [3,3,3,0] # the right operands: b1,b2,b3,b5
        c_coef = [9,27,0,0] # the output: c1,c2,c3,c5
        fa = P(list(Ai*vector(F17, a_coef)))
        fb = P(list(Ai*vector(F17, b_coef)))
        fc = P(list(Ai*vector(F17, c_coef)))
        # solution polynomials are fa,fb,fc
        # round 1
        # target polynomial that vanishes at root of unity 
        Z = x**4-1
        # random blinding values b1-b9 = [7,4,11,12,16,2]
        a = (bld1*x+bld2)*Z+fa
        b = (bld3*x+bld4)*Z+fb
        c = (bld5*x+bld6)*Z+fc
        ### choose s=2
        a_s = ZZ(a(s))*G
        b_s = ZZ(b(s))*G
        c_s = ZZ(c(s))*G
        p = (a_s,b_s,c_s)
        # round 2
        ### compute permutation challenges (beta,gamma) as hash of the transcript
        #for simplicity
        acc=1 #accumulator
        for i in range(4): #TODO parameterize number of gates
            nominator = ((a(w[i])+beta*w[i]+gamma)*(b(w[i])+beta*k1*w[i]+gamma)*(c(w[i])+beta*k2*w[i]+gamma))
            denominator = ((a(w[i])+beta*sa(w[i])+gamma)*(b(w[i])+beta*sb(w[i])+gamma)*(c(w[i])+beta*sc(w[i])+gamma))
            acc = acc*nominator/denominator
        acc=P(list(Ai*vector(F17,[1,12,10,1]))) #TODO parameterize
        Zx = (bld7*x**bld8+bld9*x+7)*Z+acc
        Z_s = ZZ(Zx(s))*G
        # round 3
        L=P(list(Ai*vector(F17, [1,0,0,0]))) #TODO parameterize
        t1Z = a*b*qm+a*ql+b*qr+c*qo+qc
        t2Z = (a+beta*x+gamma)*(b+k1*beta*x+gamma)*(c+k2*beta*x+gamma)*Zx*alpha
        Zw = Zx(w[1]*x)
        t3Z= -(a+beta*sa+gamma)*(b+beta*sb+gamma)*(c+beta*sc+gamma)*Zw*alpha
        t4Z = (Zx-1)*L*alpha**2
        tZ = t1Z+t2Z+t3Z+t4Z
        ## if everything goes well, then the following should divide without remainders
        t=P(tZ/Z)
        t_list = t.list()
        t_lo=t_list[0:6]
        t_mid=t_list[6:12]
        t_hi=t_list[12:]
        t_lo_s=ZZ(P(t_lo)(s))*G
        t_mid_s=ZZ(P(t_mid)(s))*G
        t_hi_s=ZZ(P(t_hi)(s))*G
        t_lo_s,t_mid_s,t_hi_s
        # Round 4
        a_=a(zeta)
        b_=b(zeta)
        c_=c(zeta)
        sa_=sa(zeta)
        sb_=sb(zeta)
        t_=t(zeta)
        zw_=Zx(zeta*w[1])
        l_=L(zeta)
        r1 = a_*b_*qm+a_*ql+b_*qr+c_*qo+qc
        r2 = ((a_+beta*zeta+gamma)*(b_+beta*k1*zeta+gamma)*(c_+beta*k2*zeta+gamma)*Zx)*alpha
        r3=-(a_+beta*sa_+gamma)*(b_+beta*sb_+gamma)*beta*zw_*sc*alpha
        r4=Zx*l_*alpha**2
        r=r1+r2+r3+r4
        r_=r(zeta)
        # Round 5
        v1 = P(t_lo)
        v2=zeta**6*P(t_mid)
        v3=zeta**12*P(t_hi)
        v4=-t_
        v5 = vega*(r-r_)+vega**2*(a-a_)+vega**3*(b-b_)+vega**4*(c-c_)+vega**5*(sa-sa_)+vega**6*(sb-sb_)
        W=v1+v2+v3+v4+v5
        ## opening proof polynomial
        Wz=W/(x-zeta)
        Wzw=(Zx-zw_)/(x-zeta*w[1])
        Wz_s=ZZ(Wz(s))*G
        Wzw_s=ZZ(Wzw(s))*G
        Wz_s,Wzw_s
        proof = (a_s,b_s,c_s,Z_s,t_lo_s,t_mid_s,t_hi_s,Wz_s,Wzw_s,a_,b_,c_,sa_,sb_,r_,zw_)
        return proof

In [58]:
a_s,b_s,c_s,Z_s,t_lo_s,t_mid_s,t_hi_s,Wz_s,Wzw_s,a_,b_,c_,sa_,sb_,r_,zw_ = proving()

# verification

In [61]:
def verify():
    ## TODO (1) verify that all those points are in the curve G1
    ## TODO (2) verify that the bar values are part of Fp^7 group
    ## TODO (3) validate that w_i (roots of unity) are in F_p^l
    ### verifier processed input
    qm_s = ZZ(qm(s))*G
    ql_s = ZZ(ql(s))*G
    qr_s = ZZ(qr(s))*G
    qo_s = ZZ(qo(s))*G
    qc_s = ZZ(qc(s))*G
    sa_s = ZZ(sa(s))*G
    sb_s = ZZ(sb(s))*G
    sc_s = ZZ(sc(s))*G
    ## (4) compute challenges beta, gamma, alpha, zeta, veta, upsilon as in prover description from the common inputs, public input, and elements of zk-snark
    epsilon=4 # only this is missing
    ## (5) ZERO POLYNOMIAL EVALUATION
    Z_z=F17(zeta**4-1)
    ## (6) lagrange polynomial evaluation 
    L1_z=F17((zeta**4-1)/(4*(zeta-1)))
    ## TODO(7) public input polynomial evaluation 
    ## quotient polynomial evaluation
    t_=(r_-((a_+beta*sa_+gamma)*(b_+beta*sb_+gamma)*(c_+gamma)*zw_)*alpha-L1_z*alpha**2)/Z_z;t_
    ## (9) part of batched opynomimal commitment
    d1=ZZ(a_*b_*vega)*qm_s+ZZ(a_*vega)*ql_s+ZZ(b_*vega)*qr_s+ZZ(c_*vega)*qo_s+vega*qc_s
    d2=ZZ(((a_+beta*zeta+gamma)*(b_+beta*k1*zeta+gamma)*(c_+beta*k2*zeta+gamma)*alpha*vega)+L1_z*alpha**2*vega+F17(epsilon))*Z_s
    d3=-ZZ((a_+beta*sa_+gamma)*(b_+beta*sb_+gamma)*alpha*vega*beta*zw_)*sc_s
    d=d1+d2+d3;d
    ## (10) full batched polynomial commitment [F]1
    f=t_lo_s+zeta**6*t_mid_s+zeta**12*t_hi_s+d+vega**2*a_s+vega**3*b_s+vega**4*c_s+vega**5*sa_s+vega**6*sb_s
    ## (11) group-encoded batch evaluation [E]1
    e=ZZ((t_+vega*r_+vega**2*a_+vega**3*b_+vega**4*c_+vega**5*sa_+vega**6*sb_+epsilon*zw_))*G
    ## (12)
    x1=Wz_s+epsilon*Wzw_s
    x2=s*G2
    y1=zeta*Wz_s+ZZ(epsilon*zeta*w[1])*Wzw_s+f-e
    y2=G2
    x1_=E2(x1)
    x2_=E2(x2)
    y1_=E2(y1)
    y2_=E2(y2)
    return x1_.weil_pairing(x2_,17)==y1_.weil_pairing(y2_,17)


In [62]:
assert(verify())

### references
[1] https://eprint.iacr.org/2019/953.pdf

[2] https://research.metastate.dev/plonk-by-hand-part-1/

[3] https://research.metastate.dev/plonk-by-hand-part-2-the-proof/
